In [79]:
import PyPDF2
import PySimpleGUI as sg
from pathlib import Path
import logging
import re
import sys, getopt


# TO DO
* [x] wrap **everything** in a `try`
* [ ] add gui
* [ ] test against MAP results
* [x] add original filename to output
* [ ] pyinstaller

In [19]:
def get_breaks(file):
    file = Path(file).expanduser().absolute()
    
    try:
        source_pdf = PyPDF2.PdfFileReader(str(file), False)
    except (FileNotFoundError, PyPDF2.utils.PdfReadError, IsADirectoryError) as e:
        logging.error(f'Failed to read source pdf: {e}')
        return []
    
    page_breaks = []
    
    last_record = {'id': None, 'pages': []}
    
    for page in range(0, source_pdf.numPages):
        this_record = ({'id': None, 'pages': []})
        text = source_pdf.getPage(page).extractText()
        student_number = re.search('student\W?id:\W?(\d{5,})', text, re.IGNORECASE)
        
        if student_number:
            page_breaks.append(last_record)
            this_record['id'] = student_number.group(1)
            this_record['pages'].append(page)
            last_record = this_record
            
        else:
            last_record['pages'].append(page)
            
    return page_breaks
            
        

In [55]:
def split_pdf(file, page_breaks):
    file = Path(file).expanduser().absolute()
    dest_path = file.parent/f'{file.name}_split'
    
    output = []
    
    try:
        source_pdf = PyPDF2.PdfFileReader(str(file))
    except (FileNotFoundError, PyPDF2.utils.PdfReadError, IsADirectoryError) as e:
        logging.error(f'Failed to split source pdf: {e}')
        return False
    
    for record in page_breaks:
        if not record['id']:
            continue
        pdf_writer = PyPDF2.PdfFileWriter()
        for page in record['pages']:
            pdf_writer.addPage(source_pdf.getPage(page))
        
        try:
            output_path = dest_path/f'{record["id"]}.pdf'
            output_path.parent.mkdir(exist_ok=True)
            with open(str(output_path), "wb") as dest_file:
                pdf_writer.write(dest_file)
            output.append(output_path)
        except Exception as e:
            logging.error(f'error outputing pdf: {e}')
            pass
        
    return output
    

In [78]:
def main():
    
    interactive = False
    
    if (len(sys.argv) == 1) or ('-f' in sys.argv):
        interactive = True
        fname = sg.popup_get_file('Choose a PDF to split')
    else:
        fname = sys.argv[1]

    while True:
        if not fname and interactive:
            sg.popup('Cancel', "No PDF supplied")
        else:
            
        


interactive


In [76]:
if __name__ == "__main__":
    main()

interactive


In [71]:
'-f' in sys.argv

True

In [60]:
sys.argv

['/Users/aaronciuffo/.local/share/virtualenvs/pyPDF_split-aG0Oa909/lib/python3.8/site-packages/ipykernel_launcher.py',
 '-f',
 '/Users/aaronciuffo/Library/Jupyter/runtime/kernel-5fb7074d-9a0b-4d17-b091-7941fcd2de2a.json']